In [ ]:
#import all the libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.random as rn
import time
import statistics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from numpy import array
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LeakyReLU

from keras import backend as K
from keras.callbacks import History
from keras import optimizers
from keras.layers import Dropout
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

# change the version of proxies
version='v5.1'
version='v4'

# define paths to various directories



keras_model_path=r"C:\Users\Keras-Model\49v5.1\\"
keras_model_path=r"C:\Users\Keras-Model\49v4\\"
#keras_model_path=r"C:\Users\Arpita\OneDrive - NTNU\SPM\Code\Keras-Model\\"
database_path=r"C:\Users\Database\\"
figure_path=r"C:\Users\Results\\"



#load scaler function and keras models

scalerX_fopr='scalerX_fopr_49'+version+'.gz'
scalerY_fopr='scalerY_fopr_49'+version+'.gz'
scalerX_fwpr="scalerX_fwpr_49"+version+'.gz'
scalerY_fwpr='scalerY_fwpr_49'+version+'.gz'
scalerX_fwir="scalerX_fwir_49"+version+'.gz'
scalerY_fwir='scalerY_fwir_49'+version+'.gz'


model_fopr=load_model(keras_model_path+"fopr_Olymp49-"+version+"_minmax.keras")
model_fwpr=load_model(keras_model_path+"fwpr_Olymp49-"+version+"_minmax.keras")
model_fwir=load_model(keras_model_path+"fwir_Olymp49-"+version+"_minmax.keras")   

import joblib
scalerX_fopr=joblib.load(keras_model_path+scalerX_fopr)
scalerY_fopr=joblib.load(keras_model_path+scalerY_fopr)
scalerX_fwpr=joblib.load(keras_model_path+scalerX_fwpr)
scalerY_fwpr=joblib.load(keras_model_path+scalerY_fwpr)
scalerX_fwir=joblib.load(keras_model_path+scalerX_fwir)
scalerY_fwir=joblib.load(keras_model_path+scalerY_fwir)
   
#define NPV function 
          
def NPV(Rate_Oil, Rate_Inj, Rate_Water, Oil_Price, Cost_wi, Cost_wp, disc):
#Oil price ($/bbl)60
#Cost produced water ($/bbl)6
#Cost injected water ($/bbl)2
#Annual discount factor
#oil rate in Scm3/day
#Cum_Oil is array of cummulative oil produced over the lifetime of reservoir
    NPV_calc=0
    for i in range(Rate_Oil.shape[0]):
        NPV_calc+= (Rate_Oil[i,0]*Oil_Price+Rate_Water[i,0]*Cost_wp+Rate_Inj[i,0]*Cost_wi)*365*6.29/(np.power((1+disc),i))
    return NPV_calc






In [ ]:

# state GA control parameter settings

pop=20
gen=500
CO=25
MU=75

max_BHP_prod=150
min_BHP_inj=160
max_BHP_inj=280
iteration=20

# Define a function to generate population
def generate_population(population_size, I_bound,P_bound):
    min_BHP_prod,max_BHP_prod=P_bound
    min_BHP_inj,max_BHP_inj=I_bound
    nr=20
    population=[]
    for i in range(population_size):
        individual = {
            "BHPI1": rn.uniform(min_BHP_inj,max_BHP_inj,nr),
            "BHPI2": rn.uniform(min_BHP_inj,max_BHP_inj,nr),
            "BHPI3": rn.uniform(min_BHP_inj,max_BHP_inj,nr),
            "BHPI4": rn.uniform(min_BHP_inj,max_BHP_inj,nr),
            "BHPI5": rn.uniform(min_BHP_inj,max_BHP_inj,nr),
            "BHPI6": rn.uniform(min_BHP_inj,max_BHP_inj,nr),
            "BHPP1": rn.uniform(min_BHP_prod,max_BHP_prod,nr),
            "BHPP10": rn.uniform(min_BHP_prod,max_BHP_prod,nr),
            "BHPP2": rn.uniform(min_BHP_prod,max_BHP_prod,nr),
            "BHPP3": rn.uniform(min_BHP_prod,max_BHP_prod,nr),
            "BHPP4": rn.uniform(min_BHP_prod,max_BHP_prod,nr),
            "BHPP5": rn.uniform(min_BHP_prod,max_BHP_prod,nr),
            "BHPP6": rn.uniform(min_BHP_prod,max_BHP_prod,nr),
            "BHPP7": rn.uniform(min_BHP_prod,max_BHP_prod,nr),
            "BHPP8": rn.uniform(min_BHP_prod,max_BHP_prod,nr),
            "BHPP9": rn.uniform(min_BHP_prod,max_BHP_prod,nr),



        }
        population.append(individual)
    return population

# Apply function is used to calulcate NPV for each individuals of generated population
def apply_function(individual):
    timestep= np.arange(1,21,1)
    X_new=np.vstack((timestep,individual["BHPI1"],individual["BHPI2"],\
                   individual["BHPI3"],individual["BHPI4"],individual["BHPI5"],\
                   individual["BHPI6"],individual["BHPP1"],individual["BHPP10"],individual["BHPP2"],\
                   individual["BHPP3"],individual["BHPP4"],individual["BHPP5"],individual["BHPP6"],\
                    individual["BHPP7"],individual["BHPP8"],individual["BHPP9"]))
    #print(X_new)
    X_new=X_new.T
    
    #scalerX=MinMaxScaler().fit(X_new)
    #scalerY=MinMaxScaler().fit(Y_new)
    test_preds_fopr_ga=model_fopr.predict(scalerX_fopr.transform(X_new))
    test_preds_fopr_ga=scalerY_fopr.inverse_transform(test_preds_fopr_ga)
    test_preds_fwpr_ga=model_fwpr.predict(scalerX_fwpr.transform(X_new))
    test_preds_fwpr_ga=scalerY_fwpr.inverse_transform(test_preds_fwpr_ga)
    test_preds_fwir_ga=model_fwir.predict(scalerX_fwir.transform(X_new))
    test_preds_fwir_ga=scalerY_fwir.inverse_transform(test_preds_fwir_ga)
    NPV_test=NPV(test_preds_fopr_ga,test_preds_fwir_ga,test_preds_fwpr_ga,60,-2,-6,0.08)
    #print(NPV_test)
    return NPV_test


def choice_by_roulette(sorted_population, fitness_sum):
    offset=0
    normalized_fitness_sum= fitness_sum
    lowest_fitness=apply_function(sorted_population[0])
    if lowest_fitness < 0:
        offset= -lowest_fitness
        normalized_fitness_sum += offset * len(sorted_population)

    draw=rn.uniform(0,1)

    accumulated= 0

    for individual in sorted_population:
        fitness=apply_function(individual) + offset
        probability = fitness / normalized_fitness_sum
        accumulated+= probability

        if draw <= accumulated :
            return individual


def sort_population_by_fitness(population):
       return sorted(population, key=apply_function)
#def sort_population_by_fitness(population):
 #   for individual in population:
        

def crossover(individual_a, individual_b):
    #print(individual_a["BHPI1"])
    BHPI1a=individual_a["BHPI1"]; BHPI1b=individual_b["BHPI1"]
    BHPI2a=individual_a["BHPI2"]; BHPI2b=individual_b["BHPI2"]
    BHPI3a=individual_a["BHPI3"]; BHPI3b=individual_b["BHPI3"]
    BHPI4a=individual_a["BHPI4"]; BHPI4b=individual_b["BHPI4"]
    BHPI5a=individual_a["BHPI5"]; BHPI5b=individual_b["BHPI5"]
    BHPI6a=individual_a["BHPI6"]; BHPI6b=individual_b["BHPI6"]
    BHPP1a=individual_a["BHPP1"]; BHPP1b=individual_b["BHPP1"]
    BHPP10a=individual_a["BHPP10"]; BHPP10b=individual_b["BHPP10"]
    BHPP2a=individual_a["BHPP2"]; BHPP2b=individual_b["BHPP2"]
    BHPP3a=individual_a["BHPP3"]; BHPP3b=individual_b["BHPP3"]
    BHPP4a=individual_a["BHPP4"]; BHPP4b=individual_b["BHPP4"]
    BHPP5a=individual_a["BHPP5"]; BHPP5b=individual_b["BHPP5"]
    BHPP6a=individual_a["BHPP6"]; BHPP6b=individual_b["BHPP6"]
    BHPP7a=individual_a["BHPP7"]; BHPP7b=individual_b["BHPP7"]
    BHPP8a=individual_a["BHPP8"]; BHPP8b=individual_b["BHPP8"]
    BHPP9a=individual_a["BHPP9"]; BHPP9b=individual_b["BHPP9"]

    return { "BHPI1": (BHPI1a+BHPI1b)/2, "BHPI2": (BHPI2a+BHPI2b)/2,"BHPI3": (BHPI3a+BHPI3b)/2,"BHPI4": (BHPI4a+BHPI4b)/2,\
            "BHPI5": (BHPI5a+BHPI5b)/2, "BHPI6": (BHPI6a+BHPI6b)/2,"BHPP1": (BHPP1a+BHPP1b)/2, "BHPP10": (BHPP10a+BHPP10b)/2,"BHPP2": (BHPP2a+BHPP2b)/2,\
            "BHPP3": (BHPP3a+BHPP3b)/2,"BHPP4": (BHPP4a+BHPP4b)/2,"BHPP5": (BHPP5a+BHPP5b)/2,\
            "BHPP6": (BHPP6a+BHPP6b)/2,"BHPP7": (BHPP7a+BHPP7b)/2,"BHPP8": (BHPP8a+BHPP8b)/2, "BHPP9": (BHPP9a+BHPP9b)/2}
    


def mutate(individual):
    min_prod,max_prod=(80,150)
    min_inj,max_inj=(200,260)
    min_BHP_prod=np.full(20, min_prod)
    min_BHP_inj=np.full(20, min_inj)
    max_BHP_prod=np.full(20, max_prod)
    max_BHP_inj= np.full(20, max_inj)
    a=-20
    b=20
    nr=20
    next_BHPI1=individual["BHPI1"]+rn.uniform(a,b,nr)
    next_BHPI2=individual["BHPI2"]+rn.uniform(a,b,nr)
    next_BHPI3=individual["BHPI3"]+rn.uniform(a,b,nr)
    next_BHPI4=individual["BHPI4"]+rn.uniform(a,b,nr)
    next_BHPI5=individual["BHPI5"]+rn.uniform(a,b,nr)
    next_BHPI6=individual["BHPI6"]+rn.uniform(a,b,nr)
    next_BHPP1=individual["BHPP1"]+rn.uniform(a,b,nr)
    next_BHPP10=individual["BHPP10"]+rn.uniform(a,b,nr)
    next_BHPP2=individual["BHPP2"]+rn.uniform(a,b,nr)
    next_BHPP3=individual["BHPP3"]+rn.uniform(a,b,nr)
    next_BHPP4=individual["BHPP4"]+rn.uniform(a,b,nr)
    next_BHPP5=individual["BHPP5"]+rn.uniform(a,b,nr)
    next_BHPP6=individual["BHPP6"]+rn.uniform(a,b,nr)
    next_BHPP7=individual["BHPP7"]+rn.uniform(a,b,nr)
    next_BHPP8=individual["BHPP8"]+rn.uniform(a,b,nr)
    next_BHPP9=individual["BHPP9"]+rn.uniform(a,b,nr)
    
    #Guarantee we keep inside boundaries
    next_BHPI1=np.minimum(np.maximum(next_BHPI1,min_BHP_inj), max_BHP_inj)
    next_BHPI2=np.minimum(np.maximum(next_BHPI2,min_BHP_inj), max_BHP_inj)
    next_BHPI3=np.minimum(np.maximum(next_BHPI3,min_BHP_inj), max_BHP_inj)
    next_BHPI4=np.minimum(np.maximum(next_BHPI4,min_BHP_inj), max_BHP_inj)
    next_BHPI5=np.minimum(np.maximum(next_BHPI5,min_BHP_inj), max_BHP_inj)
    next_BHPI6=np.minimum(np.maximum(next_BHPI6,min_BHP_inj), max_BHP_inj)
    next_BHPP1=np.minimum(np.maximum(next_BHPP1,min_BHP_prod), max_BHP_prod)
    next_BHPP10=np.minimum(np.maximum(next_BHPP10,min_BHP_prod), max_BHP_prod)
    next_BHPP2=np.minimum(np.maximum(next_BHPP2,min_BHP_prod), max_BHP_prod)
    next_BHPP3=np.minimum(np.maximum(next_BHPP3,min_BHP_prod), max_BHP_prod)
    next_BHPP4=np.minimum(np.maximum(next_BHPP4,min_BHP_prod), max_BHP_prod)
    next_BHPP5=np.minimum(np.maximum(next_BHPP5,min_BHP_prod), max_BHP_prod)
    next_BHPP6=np.minimum(np.maximum(next_BHPP6,min_BHP_prod), max_BHP_prod)
    next_BHPP7=np.minimum(np.maximum(next_BHPP7,min_BHP_prod), max_BHP_prod)
    next_BHPP8=np.minimum(np.maximum(next_BHPP8,min_BHP_prod), max_BHP_prod)
    next_BHPP9=np.minimum(np.maximum(next_BHPP9,min_BHP_prod), max_BHP_prod)

    
    return {"BHPI1": next_BHPI1, "BHPI2": next_BHPI2,"BHPI3": next_BHPI3,"BHPI4": next_BHPI4,\
            "BHPI5": next_BHPI5, "BHPI6": next_BHPI6,"BHPP1": next_BHPP1, "BHPP10":next_BHPP10,"BHPP2": next_BHPP2,\
            "BHPP3": next_BHPP3,"BHPP4": next_BHPP4,"BHPP5": next_BHPP5,\
            "BHPP6": next_BHPP6,"BHPP7": next_BHPP7,"BHPP8": next_BHPP8, "BHPP9": next_BHPP9}

def make_next_generation(previous_population):
    next_generation=[]
    sorted_by_fitness_population = sort_population_by_fitness(previous_population)
    population_size=len(previous_population)
    fitness_sum=sum(apply_function(individual) for individual in previous_population)
    #print(fitness_sum)
    next_generation.append(sorted_by_fitness_population[-1])
    print(next_generation)
    for i in range(population_size-1):
        first_choice=choice_by_roulette(sorted_by_fitness_population,fitness_sum)
        #print(first_choice)
        second_choice=choice_by_roulette(sorted_by_fitness_population,fitness_sum)
        #print(second_choice)

        if rn.randint(0,100) < CO:
            individual=crossover(first_choice,second_choice)
            #print(individual)
        else: individual= first_choice

        if rn.randint(0,100) < MU:
            individual = mutate(individual)
        next_generation.append(individual)
        
    return next_generation




generations= gen

population=generate_population(population_size=pop, I_bound= (200,250), P_bound=(100,150))

start=time.time()
i=1
NPV_step=[]

while True:

    print(f"GENERATION {i}")
  

    best_individual = sort_population_by_fitness(population)[-1]
    best_npv=apply_function(best_individual)
    NPV_step.append(best_npv)
    print("{:.3e}".format(best_npv))

    if i== generations:
        break
    i +=1

    population=make_next_generation(population)

print("\n BEST RESULT")
print(best_individual, "{:.3e}".format(best_npv))

end=time.time()
hours, rem=divmod(end-start, 3600)
minutes, seconds =divmod(rem, 60)
print("Time elapsed: ", int(hours), "hours", int(minutes), "minutes", int(seconds), "seconds")
np.save('NPV_sepgen500'+version+'.npy', NPV_step)

plt.figure()
plt.title('Best NPV')
plt.xlabel('Generation')
plt.ylabel('NPV $')
plt.plot(np.arange(1,generations+1,1), NPV_step[:],label='Best NPV', color='black', linewidth=3)
plt.legend(loc='lower right')
plt.savefig('Best NPV Olymp49'+version+'.pdf')
plt.show()

## Saving BHP values into csv file

In [ ]:
import pandas as pd
BHP=pd.DataFrame.from_dict(best_individual)
BHP.head()

BHP.to_csv(database_path+version+".csv")

# Schedule File Generators

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from skopt.sampler import Lhs
from smt.sampling_methods import LHS
from pyDOE import *
import os

simulation_period = 20 * 365     # days
timestep = 365                    # days
nr_inj = 6
nr_prod = 10
BHP=pd.read_csv(database_path+version+".csv")
BHP=BHP[["BHPI1", "BHPI2", "BHPI3", "BHPI4", "BHPI5", "BHPI6","BHPP1", "BHPP2", "BHPP3","BHPP4", "BHPP5", \
                                    "BHPP6", "BHPP7","BHPP8", "BHPP9", "BHPP10"]]
WELLSPEC_content =     "WELSPECS" + "\n" \
                        +"P1 P 113 78 1* OIL 1* 1* 1* 1* " + " /\n" \
                        +"P2 P 106 97 1* OIL 1* 1* 1* 1* " + " /\n" \
                        +"P3 P 110 118 1* OIL 1* 1* 1* 1* " + " /\n" \
                        +"P4 P 89 69 1* OIL 1* 1* 1* 1* " + "/\n" \
                        +"P5 P 43 82 1* OIL 1* 1* 1* 1* " + "/\n" \
                        +"P6 P 61 119 1* OIL 1* 1* 1* 1* " + "/\n" \
                        +"P7 P 83 76 1* OIL 1* 1* 1* 1* " + "/\n" \
                        +"P8 P 103 57 1* OIL 1* 1* 1* 1* " + "/\n" \
                        +"P9 P 81 69 1* OIL 1* 1* 1* 1* " + "/\n" \
                        +"P10 P 60 80 1* OIL 1* 1* 1* 1* " + "/\n" \
                        +"I1 I 65 61 1* WATER 1* 1* 1* 1* " + "/\n" \
                        +"I2 I 52 70 1* WATER 1* 1* 1* 1* " + "/\n" \
                        +"I3 I 80 81 1* WATER 1* 1* 1* 1* " + "/\n" \
                        +"I4 I 101 105 1* WATER 1* 1* 1* 1* " + "/\n" \
                        +"I5 I 90 87 1* WATER 1* 1* 1* 1* " + "/\n" \
                        +"I6 I 99 33 1* WATER 1* 1* 1* 1* " + "/\n" + "/\n"
COMPDAT_content=  "COMPDAT" +"\n"\
                    +  "P1 113 78 1 1 OPEN 1* 26.5373 0.381 1* 1* 1* Z"+  "/\n" \
                    + "P1 113 78 2 2 OPEN 1* 23.175 0.381 1* 1* 1* Z" +  "/\n" \
                    + "P1 113 78 3 3 OPEN 1* 22.9661 0.381 1* 1* 1* Z" +  "/\n" \
                    + "P2 106 97 1 1 OPEN 1* 7.73791 0.381 1* 1* 1* Z" +  "/\n" \
                    + "P2 106 97 2 2 OPEN 1* 0.000698927 0.381 1* 1* 1* Z" +  "/\n" \
                    + "P2 106 97 3 3 OPEN 1* 0.000733044 0.381 1* 1* 1* Z" +  "/\n" \
                    + "P2 106 97 4 4 OPEN 1* 1.51913 0.381 1* 1* 1* Z" +  "/\n" \
                    + "P3 110 118 1 1 OPEN 1* 23.243 0.381 1* 1* 1* Z" +  "/\n" \
                    + "P3 110 118 2 2 OPEN 1* 24.9323 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P3 110 118 3 3 OPEN 1* 20.4809 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P3 110 118 4 4 OPEN 1* 3.46789 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P4 89 69 1 1 OPEN 1* 18.9074 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P4 89 69 2 2 OPEN 1* 29.825 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P4 89 69 3 3 OPEN 1* 28.1259 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P4 89 69 4 4 OPEN 1* 0.716607 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P5 43 82 1 1 OPEN 1* 0.000794104 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P5 43 82 2 2 OPEN 1* 0.000807123 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P5 43 82 3 3 OPEN 1* 0.000710183 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P5 43 82 4 4 OPEN 1* 0.000141873 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P6 61 119 1 1 OPEN 1* 21.7062 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P6 61 119 2 2 OPEN 1* 21.6681 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P6 61 119 3 3 OPEN 1* 0.00079169 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P6 61 119 4 4 OPEN 1* 0.00015997 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P7 83 76 1 1 OPEN 1* 21.7851 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P7 83 76 2 2 OPEN 1* 25.3355 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P7 83 76 3 3 OPEN 1* 29.9869 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P7 83 76 4 4 OPEN 1* 4.85635 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P8 103 57 1 1 OPEN 1* 26.4249 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P8 103 57 2 2 OPEN 1* 37.2177 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P8 103 57 3 3 OPEN 1* 31.4823 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P8 103 57 4 4 OPEN 1* 1.98942 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P9 81 69 1 1 OPEN 1* 15.0825 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P9 81 69 2 2 OPEN 1* 24.0637 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P9 81 69 3 3 OPEN 1* 33.2584 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P9 81 69 4 4 OPEN 1* 10.8409 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P10 60 80 1 1 OPEN 1* 24.3689 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P10 60 80 2 2 OPEN 1* 30.2098 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P10 60 80 3 3 OPEN 1* 1.47486 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "P10 60 80 4 4 OPEN 1* 8.27338 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I1 65 61 1 1 OPEN 1* 18.4087 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I1 65 61 2 2 OPEN 1* 22.9074 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I1 65 61 3 3 OPEN 1* 27.8736 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I1 65 61 4 4 OPEN 1* 0.000168686 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I2 52 70 1 1 OPEN 1* 23.3738 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I2 52 70 2 2 OPEN 1* 29.3302 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I2 52 70 3 3 OPEN 1* 22.8814 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I2 52 70 4 4 OPEN 1* 4.32586 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I3 80 81 1 1 OPEN 1* 16.2781 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I3 80 81 2 2 OPEN 1* 12.6927 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I3 80 81 3 3 OPEN 1* 29.511 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I3 80 81 4 4 OPEN 1* 5.07279 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I4 101 105 1 1 OPEN 1* 21.624 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I4 101 105 2 2 OPEN 1* 22.9871 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I4 101 105 3 3 OPEN 1* 23.1498 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I4 101 105 4 4 OPEN 1* 0.000133638 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I5 90 87 1 1 OPEN 1* 23.8866 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I5 90 87 2 2 OPEN 1* 22.4762 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I5 90 87 3 3 OPEN 1* 31.4726 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I5 90 87 4 4 OPEN 1* 3.67169 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I6 99 33 1 1 OPEN 1* 28.5749 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I6 99 33 2 2 OPEN 1* 30.1634 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I6 99 33 3 3 OPEN 1* 33.4824 0.381 1* 1* 1* Z"  +  "/\n" \
                    + "I6 99 33 4 4 OPEN 1* 4.40825 0.381 1* 1* 1* Z"  +  "/\n" +"/\n"\
        
                    


name = r"C:\Users\OLYMPUS_49\OLYMPUS\schedule\OLYMPUS_SCH_49"+version +".INC"

nr_data_sets = int(simulation_period / timestep)

nr_inj=6
nr_prod=10

WCONINJE_content = []


for t in range(nr_data_sets):
    #WCONINJE_content.append("WCONINJE\n")
    for n in range(nr_inj):
            WCONINJE_content.append("WCONINJE\n")
            text = "I"+ str(n+1) + " WATER OPEN BHP " + " 800 1* " +str(BHP.iloc[t,n]) + " /\n" \
         
            WCONINJE_content.append(text)
            WCONINJE_content.append("/\n")
            #WCONINJE_content.append("\n")
            #WCONINJE_content.append("/\n")
            WCONINJE_content.append("\n")

    for z in range(nr_prod):
            WCONINJE_content.append("WCONPROD\n")
            text = "P"+ str(z+1) + " OPEN BHP " + " 1* 1* 1* 1* 1* " +str(BHP.iloc[t,z+6]) \
                   + " /\n"
            WCONINJE_content.append(text)
            WCONINJE_content.append("/\n")

    WCONINJE_content.append("\n")
    WCONINJE_content.append("TSTEP\n")
    WCONINJE_content.append(str(timestep) + "\n")
    WCONINJE_content.append("/\n")
    WCONINJE_content.append("\n")



    os.chdir(database_path)
    file = open(name, "w")
    file.writelines(WELLSPEC_content)
    file.writelines(COMPDAT_content)
    file.writelines(WCONINJE_content)
    file.close()


# Data File Generator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skopt.sampler import Lhs
from smt.sampling_methods import LHS
from pyDOE import *
import os

os.chdir(r"C:\Users\OLYMPUS_49")

name = "OLYMPUS_49"+version +".DATA"
#os.chdir(dir_schedule_file)
fin = open("OLYMPUS_49.DATA", "r")
fout= open(name, "w")
for line in fin:
    fout.write(line.replace("OLYMPUS/schedule/OLYMPUS_SCH", "OLYMPUS/schedule/OLYMPUS_SCH_49"+version))
fin.close()
fout.close()